In [2]:
import pandas as pd

In [7]:
transaction_train_path = '00_data/transactions_train.csv'

transaction_train_df = pd.read_csv(transaction_train_path, index_col=['customer_id', 'article_id'], parse_dates=['t_dat'])

In [14]:
test_date = transaction_train_df[transaction_train_df['t_dat'] >= '2018-09-21'].count()
test_date.head()

t_dat               31739925
price               31739925
sales_channel_id    31739925
dtype: int64